In [1]:
'''
[ gensim version ]

'''

# -----------------------------------------------------------

'''
[ Setting MODE ]

1. development
2. production

'''

# Developmnet
# MODE = 'DEV'

# Production
MODE = 'PRO'

In [2]:
'''
[ Libraries ]

'''
# Libraries

import os 
TEMP_FOLDER = 'tmp/gensim'
RES_FOLDER = 'res'
from smart_open import open
import json

# gensim
import logging 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) 
import tempfile 
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER)) 
import warnings 
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim') 
from gensim import corpora 
import gensim 
import numpy as np 
import pandas as pd 
import csv 
import collections 
import random 
from pprint import pprint  # pretty-printer 

# tensorflow
import tqdm
import tensorflow as tf

# directories

CONTENTS_SIZE = 7
contents_files = [ os.path.join(RES_FOLDER, 'contents/data.'+str(i)) for i in range(CONTENTS_SIZE) ]
metadata_file = os.path.join(RES_FOLDER, 'metadata.json')

if not os.path.exists('tmp') :
    os.mkdir('tmp')
if not os.path.exists(TEMP_FOLDER) :
    os.mkdir(TEMP_FOLDER)
    
documents_file = os.path.join(TEMP_FOLDER, 'documents_dict.json')
words_file = os.path.join(TEMP_FOLDER, 'words_dict.json')
preprocess_words_file = os.path.join(TEMP_FOLDER, 'preprocess_words_dict.json')

# doc2vec model
doc2vec_model_file = os.path.join(TEMP_FOLDER, 'doc2vec_model.model')


# training
train_data_file = os.path.join(TEMP_FOLDER, 'train') # made by sample code
user_read_list_file = os.path.join(TEMP_FOLDER, 'user_read_list.json')

# tensorflow model
tf_model_file = os.path.join(TEMP_FOLDER, 'tf_model')

# predict file
if MODE == 'DEV' : 
    predict_file = os.path.join(RES_FOLDER, 'predict/dev.users')
else : 
    predict_file = os.path.join(RES_FOLDER, 'predict/test.users')


# recommend file
recommend_file = os.path.join(TEMP_FOLDER, 'recommend_'+MODE+'.txt')

    
import string
import re
def normalize_text(text) :
    text = text.strip()
    # Remove Punctuation
    # regex = '|'.join(string.punctuation).split(' ')[0] - invalid
    regex = "!|\"|#|\$|%|@|'|,|-|\.|:|;|<|>|_|\"|\+|\(|\)|\[|\]"
    text = re.sub(regex, '', text)
    
    return text

Folder "tmp/gensim" will be used to save temporary dictionary and corpus.


2019-07-30 12:42:08,464 : INFO : 'pattern' package not found; tag filters are not available for English
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
'''

[ Make Vaild Documents Dictionary ]
- From metadata

[ Make Word Dictionary ]

'''

documents = {}
word_dict = {}


with open(metadata_file, 'r') as f :
    for raw in f :
        line = json.JSONDecoder().decode(raw)
        # line
        '''
        # {'magazine_id': 8982, 
            'user_id': '@bookdb', 
            'title': '사진으로 옮기기에도 아까운, 리치필드 국립공원', 
            'keyword_list': ['여행', '호주', '국립공원'], 
            'display_url': 'https://brunch.co.kr/@bookdb/782', 
            'sub_title': '세상 어디에도 없는 호주 Top 10', 
            'reg_ts': 1474944427000, 
            'article_id': 782, 
            'id': '@bookdb_782'}
        '''
        # document id
        if line['id'] not in documents :
            documents[line['id']] = line
        
        # keyword
        new_keyword = []
        for keyword in line['keyword_list'] :
            if keyword not in word_dict :
                word_dict[keyword] = len(word_dict)
            
            new_keyword.append(word_dict[keyword])
            
        documents[line['id']]['keyword_list'] = new_keyword
        
        # title
        new_title = []
        for w in line['title'].split(' ') :
            
            w = normalize_text(w)
            
            # skip blank
            if w == '' : continue
            
            if w not in word_dict :
                word_dict[w] = len(word_dict)

            new_title.append(word_dict[w])
                
        documents[line['id']]['title'] = new_title
        
        # sub_title
        new_sub_title = []
        for w in line['sub_title'].split(' ') :
            w = normalize_text(w)
            
            # skip blank
            if w == '' : continue
                
            if w not in word_dict :
                word_dict[w] = len(word_dict)

            new_sub_title.append(word_dict[w])
                
        documents[line['id']]['sub_title'] = new_sub_title

# save document file
with open(documents_file, 'w') as f :
    f.write(json.JSONEncoder().encode(documents))

# save word dictionary file
with open(words_file, 'w') as f :
    f.write(json.JSONEncoder().encode(word_dict))
        
        

In [4]:
'''

[ Make word dictionary ]
- From contents/data.0 ~ 6
- Using metadata

- for gensim project, word dict cannot be updated.
  -> store all words previously
  -> preprocess_word_dict = {}
'''

# load document file
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())

# load word dictionary file
word_dict = {}
with open(words_file, 'r') as f :
    word_dict = json.JSONDecoder().decode(f.read())


# 1. word dictionary & max length of sentences

# for training gensim initialize
preprocess_word_dict = {}

# word_dict = {} : be made previously
MAX_SENTENCE_LEN = 0
for file_number in tqdm.tqdm(range(CONTENTS_SIZE)) :
    # read each file
    with open(contents_files[file_number]) as f :
        # read each line
        for raw in f :
            line = json.JSONDecoder().decode(raw)
            
            doc_id = line['id']
            # if not exists in metadata -> ignore
            if doc_id not in documents : continue
                
            morphs = line['morphs']
            MAX_SENTENCE_LEN = max(MAX_SENTENCE_LEN, len(morphs))
            
            # sentences
            for morph in morphs :
                # sentence
                for w in morph :
                    # word
                    if w not in word_dict :
                        word_dict[w] = len(word_dict)

                    if word_dict[w] not in preprocess_word_dict :
                        preprocess_word_dict[word_dict[w]] = doc_id
            
# save word dictionary file
with open(words_file, 'w') as f :
    f.write(json.JSONEncoder().encode(word_dict))
    
# save preprocess word dictionary file
with open(preprocess_words_file, 'w') as f :
    f.write(json.JSONEncoder().encode(preprocess_word_dict))
            
print(len(word_dict))
print(len(preprocess_word_dict))
print(MAX_SENTENCE_LEN)
'''
[ result ]
2649502
1946771
35056
'''

100%|██████████| 7/7 [16:42<00:00, 128.68s/it]


2649502
1946771
35056


'\n[ result ]\n2038915\n1946771\n35056\n'

In [3]:
'''
[ get doc2vec ]
- gensim parameters
'''

if MODE == 'DEV' :
    GEN_ALPHA = 0.025
    GEN_MIN_ALPHA = 0.025
    GEN_WINDOW = 3
    GEN_ITER = 1
    GEN_SIZE = 30

else :
    GEN_ALPHA = 0.025
    GEN_MIN_ALPHA = 0.025
    GEN_WINDOW = 5
    GEN_ITER = 30
    GEN_SIZE = 100

In [4]:
'''
[ Get doc2vec ]
- using gensim
'''

# load document file
documents = {}
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())

# load word dictionary file
word_dict = {}
with open(words_file, 'r') as f :
    word_dict = json.JSONDecoder().decode(f.read())

# load preprocess word dictionary file
preprocess_word_dict = {}
with open(preprocess_words_file, 'r') as f :
    preprocess_word_dict = json.JSONDecoder().decode(f.read())
    
# build a model
doc2vec_model = gensim.models.doc2vec.Doc2Vec(
    alpha=GEN_ALPHA,
    min_alpha=GEN_MIN_ALPHA,
    window=GEN_WINDOW,
    size=GEN_SIZE
)

# build pre corpus
# -> contain all wors, all doc ids
build_corpus = []

# for all documents
print('Make build_corpus - all documents ...')
for doc_id in tqdm.tqdm(documents) :
    doc = documents[doc_id]
    '''
    # 
    < before >
    {'magazine_id': 8982,
        'user_id': '@bookdb', 
        'title': '사진으로 옮기기에도 아까운, 리치필드 국립공원', 
        'keyword_list': ['여행', '호주', '국립공원'], 
        'display_url': 'https://brunch.co.kr/@bookdb/782', 
        'sub_title': '세상 어디에도 없는 호주 Top 10', 
        'reg_ts': 1474944427000, 
        'article_id': 782, 
        'id': '@bookdb_782'}
    
    < after >
    {'article_id': 782,
     'display_url': 'https://brunch.co.kr/@bookdb/782',
     'id': '@bookdb_782',
     'keyword_list': [0, 1, 2],
     'magazine_id': 8982,
     'reg_ts': 1474944427000,
     'sub_title': [7, 8, 9, 1, 10, 11],
     'title': [3, 4, 5, 6, 2],
     'user_id': '@bookdb'}
    '''
    # add tagged_document
    sentence = []
    sentence.extend( [ str(i) for i in doc['title'] ] )
    sentence.extend( [ str(i) for i in doc['sub_title'] ])
    sentence.extend( [ str(i) for i in doc['keyword_list'] ])
    build_corpus.append( gensim.models.doc2vec.TaggedDocument(sentence, [doc_id]) )
    
# for all words
print('Make build_corpus - all words ...')
for word_id in tqdm.tqdm(preprocess_word_dict) :
    '''
    preprocess_word_dict
    => { word_id : doc_id, }
    '''
    doc_id = preprocess_word_dict[word_id]
    doc = documents[doc_id]
    sentence = []
    sentence.extend( [ str(i) for i in doc['title'] ] )
    sentence.extend( [ str(i) for i in doc['sub_title'] ])
    sentence.extend( [ str(i) for i in doc['keyword_list'] ])
    sentence.append(word_id)
    build_corpus.append( gensim.models.doc2vec.TaggedDocument(sentence, [doc_id]) )
    
# momery leak problem,
# delete unnecessary dicts 
print('delete dicts')
documents = {}
word_dict = {}
preprocess_word_dict = {}

print('build_vocab start')

doc2vec_model.build_vocab( build_corpus )
%time doc2vec_model.train(build_corpus, total_examples=doc2vec_model.corpus_count, epochs=GEN_ITER)
doc2vec_model.save(doc2vec_model_file)

print('build_vocab initialize finished')

print('delete build_corpus')
build_corpus = []

W0728 04:26:01.091965 140522813556544 base_any2vec.py:723] consider setting layer size to a multiple of 4 for greater performance
  2%|▏         | 11802/643104 [00:00<00:05, 118015.05it/s]

Make build_corpus - all documents ...


  1%|          | 16094/1946771 [00:00<00:11, 160935.17it/s]

Make build_corpus - all words ...


100%|██████████| 1946771/1946771 [00:19<00:00, 100078.08it/s]
I0728 04:26:28.619967 140522813556544 doc2vec.py:1378] collecting all words and their counts
I0728 04:26:28.622622 140522813556544 doc2vec.py:1322] PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
I0728 04:26:28.696525 140522813556544 doc2vec.py:1322] PROGRESS: at example #10000, processed 94719 words (1319780/s), 34999 word types, 10000 tags
I0728 04:26:28.768733 140522813556544 doc2vec.py:1322] PROGRESS: at example #20000, processed 190632 words (1343847/s), 59739 word types, 20000 tags


...

I0728 04:28:13.917754 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 47.30% examples, 197754 words/s, in_qsize 6, out_qsize 0
I0728 04:28:14.948737 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 48.10% examples, 198223 words/s, in_qsize 6, out_qsize 0
I0728 04:28:15.949569 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 48.93% examples, 198913 words/s, in_qsize 5, out_qsize 0
I0728 04:28:16.952456 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 49.73% examples, 199440 words/s, in_qsize 5, out_qsize 0
I0728 04:28:17.965233 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 50.50% examples, 199911 words/s, in_qsize 5, out_qsize 0
I0728 04:28:18.967318 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 51.28% examples, 200255 words/s, in_qsize 5, out_qsize 0
I0728 04:28:19.988265 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 52.05% examples, 200546 words/s, in_qsize 5, out_qsize 0
I0728 04:28:21.07836

I0728 04:29:15.212774 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 92.08% examples, 205040 words/s, in_qsize 6, out_qsize 0
I0728 04:29:16.213456 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 92.92% examples, 205326 words/s, in_qsize 5, out_qsize 0
I0728 04:29:17.250777 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 93.74% examples, 205550 words/s, in_qsize 6, out_qsize 0
I0728 04:29:18.333634 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 94.56% examples, 205700 words/s, in_qsize 5, out_qsize 0
I0728 04:29:19.345872 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 95.43% examples, 206033 words/s, in_qsize 5, out_qsize 0
I0728 04:29:20.422070 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 96.31% examples, 206260 words/s, in_qsize 5, out_qsize 0
I0728 04:29:21.464042 140522813556544 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 97.19% examples, 206537 words/s, in_qsize 5, out_qsize 0
I0728 04:29:22.47851

CPU times: user 2min 48s, sys: 21.6 s, total: 3min 10s
Wall time: 2min 17s


I0728 04:29:26.233874 140522813556544 utils.py:616] storing np array 'vectors' to tmp/gensim/doc2vec_model.model.wv.vectors.npy
I0728 04:29:26.616615 140522813556544 utils.py:616] storing np array 'vectors_docs' to tmp/gensim/doc2vec_model.model.docvecs.vectors_docs.npy
/usr/local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
I0728 04:29:32.296768 140522813556544 utils.py:556] saved tmp/gensim/doc2vec_model.model


build_vocab initialize finished


In [4]:
'''
[ Get doc2vec continue ]
- using gensim

- Training Continue ...
  -> with contents/data.0 ~ 6

'''

    
print('build_vocab continue ...!')

# load document file
documents = {}
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())

# load word dictionary file
word_dict = {}
with open(words_file, 'r') as f :
    word_dict = json.JSONDecoder().decode(f.read())

# load preprocess word dictionary file
preprocess_word_dict = {}
with open(preprocess_words_file, 'r') as f :
    preprocess_word_dict = json.JSONDecoder().decode(f.read())

# continue training with the loaded model
doc2vec_model = gensim.models.Doc2Vec.load( doc2vec_model_file ) 


# continue training
SIZE_OF_LOOP = 5000
train_corpus = []

for file_number in range(CONTENTS_SIZE) :
    print('read ' + contents_files[file_number] + '...')
    # read each file
    with open(contents_files[file_number]) as f :
        count = 0
        for raw in f :
            
            line = json.JSONDecoder().decode(raw)
            doc_id = line['id']
            
            # if doc_id not in metadata -> skip
            if doc_id not in documents : continue
            
            for morphs in line['morphs'] :
                # transfer word to word_id
                morphs = [ word_dict[w] for w in morphs ]
                train_corpus.append( gensim.models.doc2vec.TaggedDocument(morphs, [doc_id]) )

            # for memery problem
            # -> train each 5000 loop
            count = count + 1
            if count >= SIZE_OF_LOOP :
                # training
                %time doc2vec_model.train(train_corpus, total_examples=doc2vec_model.corpus_count, epochs=GEN_ITER)
                
                # reset corpus
                count = 0
                train_corpus = []
                
        # finish read one file
        if count > 0 :            
            # training
            %time doc2vec_model.train(train_corpus, total_examples=doc2vec_model.corpus_count, epochs=GEN_ITER)

            # reset corpus
            count = 0
            train_corpus = []
            

print('Training Finished..!')

# save
doc2vec_model.save(doc2vec_model_file)
print('Model Saved..!')

build_vocab continue ...!


I0729 19:47:44.587479 139701344483136 utils.py:422] loading Doc2Vec object from tmp/gensim/doc2vec_model.model
/usr/local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
I0729 19:47:48.541280 139701344483136 utils.py:456] loading vocabulary recursively from tmp/gensim/doc2vec_model.model.vocabulary.* with mmap=None
I0729 19:47:48.542206 139701344483136 utils.py:456] loading trainables recursively from tmp/gensim/doc2vec_model.model.trainables.* with mmap=None
I0729 19:47:48.545166 139701344483136 utils.py:461] loading syn1neg from tmp/gensim/doc2vec_model.model.trainables.syn1neg.npy with mmap=None
I0729 19:47:49.209282 139701344483136 utils.py:456] loading wv recursively from tmp/gensim/doc2vec_

read res/contents/data.0...


I0729 19:48:10.176561 139701344483136 base_any2vec.py:1210] training model with 3 workers on 362602 vocabulary and 30 features, using sg=0 hs=0 sample=0.001 negative=5 window=3
I0729 19:48:11.232091 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 1.20% examples, 30138 words/s, in_qsize 6, out_qsize 0
I0729 19:48:12.240648 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 2.91% examples, 36958 words/s, in_qsize 5, out_qsize 0
I0729 19:48:13.251120 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 4.42% examples, 37582 words/s, in_qsize 6, out_qsize 0
I0729 19:48:14.338812 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 6.12% examples, 38337 words/s, in_qsize 5, out_qsize 0
I0729 19:48:15.376839 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 7.82% examples, 39173 words/s, in_qsize 5, out_qsize 0
I0729 19:48:16.419517 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 9.02% examples, 37589 words/s, in_qsize 5, out_qsi

I0729 19:49:11.280842 139701344483136 base_any2vec.py:1346] EPOCH - 1 : training on 5376903 raw words (2386157 effective words) took 61.1s, 39069 effective words/s
W0729 19:49:11.281810 139701344483136 base_any2vec.py:1357] EPOCH - 1 : supplied example count (2386157) did not equal expected count (2589875)
I0729 19:49:11.282892 139701344483136 base_any2vec.py:1382] training on a 5376903 raw words (2386157 effective words) took 61.1s, 39060 effective words/s


...

CPU times: user 52.3 s, sys: 16.9 s, total: 1min 9s
Wall time: 1min 1s


I0729 22:33:58.187832 139701344483136 base_any2vec.py:1210] training model with 3 workers on 362602 vocabulary and 30 features, using sg=0 hs=0 sample=0.001 negative=5 window=3
I0729 22:33:59.252776 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 1.37% examples, 33712 words/s, in_qsize 6, out_qsize 0
I0729 22:34:00.343568 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 2.89% examples, 34884 words/s, in_qsize 6, out_qsize 0
I0729 22:34:01.369964 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 4.76% examples, 38880 words/s, in_qsize 5, out_qsize 0
I0729 22:34:02.381886 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 6.13% examples, 37934 words/s, in_qsize 6, out_qsize 0
I0729 22:34:03.467887 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 7.82% examples, 38442 words/s, in_qsize 6, out_qsize 0
I0729 22:34:04.541703 139701344483136 base_any2vec.py:1305] EPOCH 1 - PROGRESS: at 9.37% examples, 38233 words/s, in_qsize 5, out_qsi

CPU times: user 23.7 s, sys: 7.72 s, total: 31.5 s
Wall time: 27.6 s
Training Finished..!


I0729 22:34:26.364196 139701344483136 utils.py:616] storing np array 'vectors' to tmp/gensim/doc2vec_model.model.wv.vectors.npy
I0729 22:34:27.065320 139701344483136 utils.py:616] storing np array 'vectors_docs' to tmp/gensim/doc2vec_model.model.docvecs.vectors_docs.npy
/usr/local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
I0729 22:34:31.986548 139701344483136 utils.py:556] saved tmp/gensim/doc2vec_model.model


Model Saved..!


In [4]:
'''
[ Make TensorFlow Model ]

- setting option

'''

if MODE == 'DEV' :
    learning_rate = 0.01
    total_epoch = 2
    n_hidden = 50
    n_step = READ_WINDOW_SIZE = 4
    n_input = n_output = 30
    

else :
    learning_rate = 0.01
    total_epoch = 35
    n_hidden = 150
    n_step = READ_WINDOW_SIZE = 20
    n_input = n_output = 100

In [5]:
'''
[ Make TensorFlow Model ]

- get read list by users

'''

# load document file
documents = {}
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())

# { user : [ docids ] }
user_read_list = {}
with open(train_data_file, 'r') as f :
    for line in f :
        line = line.split()
        user_id = line[0]
        if user_id not in user_read_list :
            user_read_list[user_id] = []
        # remove documents not included in documents_dictionary 
        user_read_list[user_id].extend([ d for d in line[1:] if d in documents ])

print(len(user_read_list))
'''
[ result ]
before except document constraint
: 296520

after
: 296520
'''

# save user_read_list
with open(user_read_list_file, 'w') as f :
    f.write(json.JSONEncoder().encode(user_read_list))

# for memory..
documents = {}

296520


In [5]:
'''
[ Make TensorFlow Model ]

'''

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])

W = tf.Variable(tf.random_normal([n_hidden, n_output]))
b = tf.Variable(tf.random_normal([n_output]))


# RNN cell
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.8)

# other cell
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# combination
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# tf.nn.dynamic_run : make NN
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
# inputs -> [ batch_size, n_step, n_input ]
# outputs -> [ batch_size, n_step, n_hidden ]

#outputs: [batch_size, n_step, n_hidden]
# -> [n_step, batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])

#-> [batch-size, n_hidden]
outputs = outputs[-1]
rnn_model = tf.add(tf.matmul(outputs, W), b, name="final_model")

'''
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=rnn_model, labels=Y))
'''
cost = tf.reduce_mean(tf.squared_difference(Y, rnn_model))


rnn_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

W0730 12:42:33.859657 140575710967616 deprecation.py:323] From <ipython-input-5-a029fb90eb7c>:14: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0730 12:42:33.868351 140575710967616 deprecation.py:323] From <ipython-input-5-a029fb90eb7c>:23: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0730 12:42:33.870426 140575710967616 deprecation.py:323] From <ipython-input-5-a029fb90eb7c>:26: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W073

In [6]:
'''
[ Training ]

'''

def get_vecs_from_docid(docid, model) :
    try :
        v = model.docvecs[docid]
    except :
        v = np.zeros(n_input)
    
    return v

        
# load document file
documents = {}
with open(documents_file, 'r') as f :
    documents = json.JSONDecoder().decode(f.read())
    
user_read_list = {}
with open(user_read_list_file, 'r') as f :
    user_read_list = json.JSONDecoder().decode(f.read())

# load doc2vec model
doc2vec_model = gensim.models.Doc2Vec.load( doc2vec_model_file )

# saver
saver = tf.train.Saver()

# session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

SIZE_OF_LOOP = 5000
count = 0

input_batch = []
target_batch = []
for user in user_read_list :
    reads = user_read_list[user]
    reads_len = len(reads)

    # empty -> skip
    if reads_len == 0 : continue
    
    # READ_WINDOW = [ t1, t2, ... , tn ], + need answer
    SEQ_SIZE = READ_WINDOW_SIZE+1
    for i in range(0, reads_len-SEQ_SIZE, SEQ_SIZE) :
        input_batch.append([ get_vecs_from_docid(d, doc2vec_model) for d in reads[i:i+READ_WINDOW_SIZE] ])
        target_batch.append( get_vecs_from_docid(reads[i+READ_WINDOW_SIZE], doc2vec_model) )
        
        # for memory problem
        count = count + 1
        if count >= SIZE_OF_LOOP :
            # training
            for epoch in range(total_epoch) :
                _, loss = sess.run([rnn_optimizer, cost],
                                      feed_dict={X: input_batch, Y: target_batch})

                print('Epoch: ', '%04d' % (epoch + 1),
                        'cost = ', '{:.6f}'.format(loss))
            input_batch = []
            target_batch = []
            count = 0
    
    # left data
    left_len = reads_len % SEQ_SIZE
    # if only have one data, skip
    if left_len == 1 : continue
    if left_len == 0 : left_len = SEQ_SIZE
    left_vec = [ np.zeros(n_input) for i in range(SEQ_SIZE - left_len) ]
    left_vec.extend([ get_vecs_from_docid(d, doc2vec_model) for d in reads[reads_len-left_len:reads_len-1] ])
    
    input_batch.append( left_vec )
    target_batch.append( get_vecs_from_docid(reads[reads_len-1], doc2vec_model) )
    
    # for memory problem
    count = count + 1
    if count >= SIZE_OF_LOOP :
        # training
        for epoch in range(total_epoch) :
            _, loss = sess.run([rnn_optimizer, cost],
                                  feed_dict={X: input_batch, Y: target_batch})

            print('Epoch: ', '%04d' % (epoch + 1),
                    'cost = ', '{:.6f}'.format(loss))
        input_batch = []
        target_batch = []
        count = 0
    
if count > 0 :
    # training
    for epoch in range(total_epoch) :
        _, loss = sess.run([rnn_optimizer, cost],
                              feed_dict={X: input_batch, Y: target_batch})

        print('Epoch: ', '%04d' % (epoch + 1),
                'cost = ', '{:.6f}'.format(loss))
    input_batch = []
    target_batch = []
    count = 0
    

print('finish optimizing..!')
# save tensorfow_model
saver.save(sess, tf_model_file, global_step=0)
print('tensorflow model saved..!')

I0730 12:42:57.413270 140575710967616 utils.py:422] loading Doc2Vec object from tmp/gensim/doc2vec_model.model
/usr/local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
I0730 12:43:02.696561 140575710967616 utils.py:456] loading vocabulary recursively from tmp/gensim/doc2vec_model.model.vocabulary.* with mmap=None
I0730 12:43:02.700071 140575710967616 utils.py:456] loading trainables recursively from tmp/gensim/doc2vec_model.model.trainables.* with mmap=None
I0730 12:43:02.703949 140575710967616 utils.py:461] loading syn1neg from tmp/gensim/doc2vec_model.model.trainables.syn1neg.npy with mmap=None
I0730 12:43:02.734260 140575710967616 utils.py:456] loading wv recursively from tmp/gensim/doc2vec_

Epoch:  0001 cost =  1.015777
Epoch:  0002 cost =  0.835134
Epoch:  0001 cost =  0.623752
Epoch:  0002 cost =  0.435558
Epoch:  0001 cost =  0.297350
Epoch:  0002 cost =  0.180002
Epoch:  0001 cost =  0.093365
Epoch:  0002 cost =  0.058391
Epoch:  0001 cost =  0.051053
Epoch:  0002 cost =  0.046277
Epoch:  0001 cost =  0.047225
Epoch:  0002 cost =  0.044684
Epoch:  0001 cost =  0.040046
Epoch:  0002 cost =  0.036637
Epoch:  0001 cost =  0.034950
Epoch:  0002 cost =  0.032163
Epoch:  0001 cost =  0.028715
Epoch:  0002 cost =  0.026315
Epoch:  0001 cost =  0.024459
Epoch:  0002 cost =  0.022815
Epoch:  0001 cost =  0.019995
Epoch:  0002 cost =  0.018398
Epoch:  0001 cost =  0.018264
Epoch:  0002 cost =  0.017415
Epoch:  0001 cost =  0.016199
Epoch:  0002 cost =  0.016531
Epoch:  0001 cost =  0.017255
Epoch:  0002 cost =  0.017432
Epoch:  0001 cost =  0.016909
Epoch:  0002 cost =  0.016342
Epoch:  0001 cost =  0.016025
Epoch:  0002 cost =  0.015179
Epoch:  0001 cost =  0.012333
Epoch:  00

Epoch:  0001 cost =  0.010903
Epoch:  0002 cost =  0.010825
Epoch:  0001 cost =  0.009960
Epoch:  0002 cost =  0.009866
Epoch:  0001 cost =  0.012712
Epoch:  0002 cost =  0.012743
Epoch:  0001 cost =  0.013849
Epoch:  0002 cost =  0.013818
Epoch:  0001 cost =  0.009591
Epoch:  0002 cost =  0.009537
Epoch:  0001 cost =  0.011686
Epoch:  0002 cost =  0.011662
Epoch:  0001 cost =  0.010452
Epoch:  0002 cost =  0.010499
Epoch:  0001 cost =  0.013185
Epoch:  0002 cost =  0.013208
Epoch:  0001 cost =  0.012017
Epoch:  0002 cost =  0.012022
Epoch:  0001 cost =  0.009481
Epoch:  0002 cost =  0.009442
Epoch:  0001 cost =  0.014354
Epoch:  0002 cost =  0.014243
Epoch:  0001 cost =  0.009553
Epoch:  0002 cost =  0.009610
Epoch:  0001 cost =  0.012852
Epoch:  0002 cost =  0.012693
Epoch:  0001 cost =  0.011666
Epoch:  0002 cost =  0.011594
Epoch:  0001 cost =  0.011321
Epoch:  0002 cost =  0.011310
Epoch:  0001 cost =  0.011298
Epoch:  0002 cost =  0.011343
Epoch:  0001 cost =  0.010073
Epoch:  00

Epoch:  0001 cost =  0.010959
Epoch:  0002 cost =  0.011031
Epoch:  0001 cost =  0.012061
Epoch:  0002 cost =  0.012049
Epoch:  0001 cost =  0.008128
Epoch:  0002 cost =  0.008077
Epoch:  0001 cost =  0.009073
Epoch:  0002 cost =  0.009096
Epoch:  0001 cost =  0.011856
Epoch:  0002 cost =  0.012017
Epoch:  0001 cost =  0.015534
Epoch:  0002 cost =  0.015401
Epoch:  0001 cost =  0.011383
Epoch:  0002 cost =  0.011420
Epoch:  0001 cost =  0.012791
Epoch:  0002 cost =  0.012761
Epoch:  0001 cost =  0.010928
Epoch:  0002 cost =  0.010902
Epoch:  0001 cost =  0.011725
Epoch:  0002 cost =  0.011762
Epoch:  0001 cost =  0.011405
Epoch:  0002 cost =  0.011388
Epoch:  0001 cost =  0.013240
Epoch:  0002 cost =  0.013190
Epoch:  0001 cost =  0.012375
Epoch:  0002 cost =  0.012317
Epoch:  0001 cost =  0.012865
Epoch:  0002 cost =  0.012831
Epoch:  0001 cost =  0.011594
Epoch:  0002 cost =  0.011675
Epoch:  0001 cost =  0.012023
Epoch:  0002 cost =  0.012051
Epoch:  0001 cost =  0.012310
Epoch:  00

Epoch:  0001 cost =  0.010197
Epoch:  0002 cost =  0.010202
Epoch:  0001 cost =  0.009892
Epoch:  0002 cost =  0.009888
Epoch:  0001 cost =  0.010947
Epoch:  0002 cost =  0.010939
Epoch:  0001 cost =  0.009944
Epoch:  0002 cost =  0.009875
Epoch:  0001 cost =  0.012471
Epoch:  0002 cost =  0.012414
Epoch:  0001 cost =  0.009856
Epoch:  0002 cost =  0.009858
Epoch:  0001 cost =  0.011862
Epoch:  0002 cost =  0.011815
Epoch:  0001 cost =  0.009584
Epoch:  0002 cost =  0.009473
Epoch:  0001 cost =  0.012317
Epoch:  0002 cost =  0.012271
Epoch:  0001 cost =  0.011954
Epoch:  0002 cost =  0.011869
Epoch:  0001 cost =  0.010359
Epoch:  0002 cost =  0.010332
Epoch:  0001 cost =  0.012617
Epoch:  0002 cost =  0.012493
Epoch:  0001 cost =  0.010703
Epoch:  0002 cost =  0.010665
Epoch:  0001 cost =  0.013563
Epoch:  0002 cost =  0.013491
Epoch:  0001 cost =  0.008778
Epoch:  0002 cost =  0.008691
Epoch:  0001 cost =  0.010840
Epoch:  0002 cost =  0.010845
Epoch:  0001 cost =  0.011360
Epoch:  00

Epoch:  0001 cost =  0.011352
Epoch:  0002 cost =  0.011295
Epoch:  0001 cost =  0.012199
Epoch:  0002 cost =  0.012214
Epoch:  0001 cost =  0.010902
Epoch:  0002 cost =  0.010855
Epoch:  0001 cost =  0.010658
Epoch:  0002 cost =  0.010640
Epoch:  0001 cost =  0.012957
Epoch:  0002 cost =  0.013005
Epoch:  0001 cost =  0.011569
Epoch:  0002 cost =  0.011547
Epoch:  0001 cost =  0.012631
Epoch:  0002 cost =  0.012633
Epoch:  0001 cost =  0.011232
Epoch:  0002 cost =  0.011270
Epoch:  0001 cost =  0.011517
Epoch:  0002 cost =  0.011510
Epoch:  0001 cost =  0.012298
Epoch:  0002 cost =  0.012333
Epoch:  0001 cost =  0.011190
Epoch:  0002 cost =  0.011178
Epoch:  0001 cost =  0.012909
Epoch:  0002 cost =  0.012917
Epoch:  0001 cost =  0.008493
Epoch:  0002 cost =  0.008468
Epoch:  0001 cost =  0.009017
Epoch:  0002 cost =  0.009003
Epoch:  0001 cost =  0.010018
Epoch:  0002 cost =  0.009967
Epoch:  0001 cost =  0.010100
Epoch:  0002 cost =  0.010103
Epoch:  0001 cost =  0.009776
Epoch:  00

Epoch:  0001 cost =  0.010106
Epoch:  0002 cost =  0.010107
Epoch:  0001 cost =  0.013814
Epoch:  0002 cost =  0.013753
Epoch:  0001 cost =  0.008014
Epoch:  0002 cost =  0.007996
Epoch:  0001 cost =  0.011481
Epoch:  0002 cost =  0.011451
Epoch:  0001 cost =  0.010122
Epoch:  0002 cost =  0.010124
Epoch:  0001 cost =  0.012059
Epoch:  0002 cost =  0.012031
Epoch:  0001 cost =  0.008962
Epoch:  0002 cost =  0.008950
Epoch:  0001 cost =  0.012958
Epoch:  0002 cost =  0.012887
Epoch:  0001 cost =  0.010761
Epoch:  0002 cost =  0.010728
Epoch:  0001 cost =  0.012013
Epoch:  0002 cost =  0.011983
Epoch:  0001 cost =  0.009670
Epoch:  0002 cost =  0.009667
Epoch:  0001 cost =  0.012814
Epoch:  0002 cost =  0.012884
Epoch:  0001 cost =  0.011561
Epoch:  0002 cost =  0.011506
Epoch:  0001 cost =  0.012510
Epoch:  0002 cost =  0.012592
Epoch:  0001 cost =  0.011407
Epoch:  0002 cost =  0.011417
Epoch:  0001 cost =  0.011918
Epoch:  0002 cost =  0.011890
Epoch:  0001 cost =  0.012280
Epoch:  00

In [15]:
'''
[ predict ]

'''

def get_vecs_from_docid(docid, model) :
    try :
        v = model.docvecs[docid]
    except :
        v = np.zeros(n_input)
    
    return v

# load user read list
user_read_list = {}
with open(user_read_list_file, 'r') as f :
    user_read_list = json.JSONDecoder().decode(f.read())

# load doc2vec model
doc2vec_model = gensim.models.Doc2Vec.load( doc2vec_model_file )

    
# load tensorflow model
tf.reset_default_graph()

sess = tf.Session()
saver = tf.train.import_meta_graph(tf_model_file+'-0.meta')
checkpoint = tf.train.get_checkpoint_state(TEMP_FOLDER)
saver.restore(sess, checkpoint.model_checkpoint_path)


prediction = tf.get_default_graph().get_tensor_by_name('final_model:0')
X = tf.get_default_graph().get_tensor_by_name('Placeholder:0')

with open(predict_file, 'r') as f :
    with open(recommend_file, 'w') as fw :
        ith = 0
        for user_id in f :
            ith = ith+1
            user_id = user_id.strip()

            print('running : '+str(ith)+'th. '+user_id)
            read_list = user_read_list.get(user_id, [])
            x = []
            if len(read_list) >= READ_WINDOW_SIZE :
                x = [ get_vecs_from_docid(d, doc2vec_model) for d in read_list[len(read_list) - READ_WINDOW_SIZE:] ]
            else :
                need = READ_WINDOW_SIZE - len(read_list)
                for i in range(need) :
                    x.append(np.zeros(n_input))
                x.extend( [ get_vecs_from_docid(d, doc2vec_model) for d in read_list ])
            
            predict = sess.run(prediction, feed_dict={X: [ x ]})
            #recommend_docs = model.docvecs.most_similar(predict, topn=100)
            recommend_docs = doc2vec_model.docvecs.most_similar(predict, topn=100+len(read_list))
            
            # remove previously seen docs
            read_list.sort()
            
            count = 0
            fw.write(user_id + ' ')
            for d in recommend_docs :
                # if already seen, skip
                if d[0] in read_list : continue
                    
                fw.write(d[0] + ' ')
                count = count+1
                
                if count >= 100 : break
                    
            fw.write('\n')

            
print('finish prediction..!')

I0730 13:30:31.729187 140575710967616 utils.py:422] loading Doc2Vec object from tmp/gensim/doc2vec_model.model
/usr/local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
I0730 13:30:35.716839 140575710967616 utils.py:456] loading vocabulary recursively from tmp/gensim/doc2vec_model.model.vocabulary.* with mmap=None
I0730 13:30:35.717785 140575710967616 utils.py:456] loading trainables recursively from tmp/gensim/doc2vec_model.model.trainables.* with mmap=None
I0730 13:30:35.718390 140575710967616 utils.py:461] loading syn1neg from tmp/gensim/doc2vec_model.model.trainables.syn1neg.npy with mmap=None
I0730 13:30:36.065876 140575710967616 utils.py:456] loading wv recursively from tmp/gensim/doc2vec_

running : 1th. #d6866a498157771069fdf15361cb012b
running : 2th. #f963fb8c5d9d14d503fc4e80bd8617b4
running : 3th. #87a6479c91e4276374378f1d28eb307c
running : 4th. #677e984e245b344f61dc5d3cc1f352c8
running : 5th. #519f45eb14e4807e8714fb7e835463eb
running : 6th. #ec0fb734ba02a29c62c64e7ac7a8f13e
running : 7th. #a0df5bd0e5a5bbc28b87f8c64462667c
running : 8th. #157f9687d223501f0e18cc469462cab1
running : 9th. #dfe07d49ef81ac7f00653b5b82b70d83
running : 10th. #dce6d42815d547618757cd3d963c112c
running : 11th. #22b1c4eb8473c4fece4f7c44f0a70860
running : 12th. #230348877e0eff8c66dfedbc57ba4af7
running : 13th. #04db42d36de8125e1c38253dd0fa29df
running : 14th. #5fa86aa0978f7db9837415ffa47de849
running : 15th. #58437f4c25bf7cab8e1aca442bf903bf
running : 16th. #01c54db768e8b7d5be15cca20b1c4fb5
running : 17th. #c41bfa2fc500bc36d92dc307a4d2a562
running : 18th. #3ea4ead3497d82697f0b8795f583fb98
running : 19th. #23d761247c9ac9bbbfb4093d78dc782d
running : 20th. #70092bb34c321ccc0dce60de16132829
running :

running : 164th. #1508877bf0e84e3c17fab2727a469196
running : 165th. #32a9d11d80ecc3b33def699fd733bce1
running : 166th. #6f89f14c2dc18f035cc23a2154b6ab5b
running : 167th. #ba0ada0c9f4f377afa41c6c90dc5ac20
running : 168th. #0cb6a6b4b9eb3ea6130673fe61d748ef
running : 169th. #1deae51d0561b870522e51ee963b4c8e
running : 170th. #c4c1e57903e6acb8d84be4cf160e9365
running : 171th. #c989b7db1864751d24b094ba665dbea4
running : 172th. #a514043bfdbbac044d55baad5fa24e87
running : 173th. #88a321c076d7134ddb11bc0919299cc9
running : 174th. #fd3a1e6204b7f53f03f2e37c605cd764
running : 175th. #f64fd9591e073fb1ea2f561c72962234
running : 176th. #88bbde131f332fd3732e66ab0d48aaaf
running : 177th. #eb2f9a8ccefb2405c6a65d201a84d435
running : 178th. #61bc21ee80aad189b0cd26a9d72f0ed1
running : 179th. #60a5ff7f886c2a31cb66067cd8b53b06
running : 180th. #71c99c346e3b0882006ecfa8b0f8dc4a
running : 181th. #de68818b743368a733b2bf8def90a054
running : 182th. #6e234811c370c5670ce6f8bd21a5ffe5
running : 183th. #62ae57432fa20

running : 328th. #62a35b1a252845ffa760aaa7654f8088
running : 329th. #4cd430ee211e29b69e21b5e60e9a93ce
running : 330th. #0f73d2ba2829b54549255287bf8fce92
running : 331th. #24e5c7566a5f1ae91f1fa02be84a8fc3
running : 332th. #d26b3fbd2ed57bd99e709ead073b5787
running : 333th. #eee516c92960581c1782cf60026c9620
running : 334th. #e3210ed29e2a1bd9237b3a804553fcd5
running : 335th. #2726eb01304b340e9776a26d9e056d67
running : 336th. #fcaf79430d685650b123167ce947bac4
running : 337th. #3654b12a6908dbb2b078ca3c9ab73554
running : 338th. #50aa70a9ae4b7c97a30393c7576d4b3b
running : 339th. #037f14bbc9e454d362f437dc9fbf1af9
running : 340th. #66e4e0a0a4921b4d235f23d8993bce42
running : 341th. #e047b24c2fc624b9827e0fd9882058ca
running : 342th. #fe75bd392b42e2d99b5fb14ebd7a65b7
running : 343th. #c0830aca17a76c20975d413e6e872784
running : 344th. #13c44f171221edafe3858c522ed1421e
running : 345th. #944532e6b16a0b04b298b511e3f80a44
running : 346th. #a081ad20d89f7bd5458dbb79ba16d057
running : 347th. #83412ec817c8d

running : 493th. #656282dc01c82c3537f24785aa2adc58
running : 494th. #821c50c1bcc3d63c866e23386a7273c6
running : 495th. #c6c73bfd33517c7d60073c8cd1ead528
running : 496th. #cd25eabc9c44fc6b681c62583a6953f7
running : 497th. #e69b26ef060ca4b9efc6b315b7dc870e
running : 498th. #36286c5e0d607d608b00dfa3c44566d4
running : 499th. #1c07a0f34f68e35da6180d50ac886ee9
running : 500th. #8865a696bc4e0758bd7d9d04ca0e2dc4
running : 501th. #2d15321890050d685eac255a64a1e5d2
running : 502th. #c56d778030227dd1244a26912f90611a
running : 503th. #7f8167d311f79c99f6dc2c8d8b064d4e
running : 504th. #258e9ae0f6292a6ba152285471d59f6f
running : 505th. #e74812e0a4acac1af1b0aa81519302ff
running : 506th. #fde8d0aaefb89022f1733de65d37a95c
running : 507th. #93c7c78cd17135ac09cd2b078a12cfd3
running : 508th. #3ccc0d00c9fa6fa302f03d5aea8806de
running : 509th. #0c4b83faa5abc0f5155c7bfa0c83e40b
running : 510th. #6fcbd560e741d0fdb6dbccaff4721e85
running : 511th. #85f8d07d421702362081b713a750f011
running : 512th. #fb888772520a8

...

running : 2262th. #87b5c55e4a36f947ae87a0d3c4c1b944
running : 2263th. #8f6be162b86829a6380d94a1ee636e40
running : 2264th. #50a6923a635665a6386cfbfb074a4d66
running : 2265th. #d00e582ea7197c53ecb037413fb02649
running : 2266th. #135a0447f17026d7611ff7d4ccb90474
running : 2267th. #1a81a295d348494cfd8b1770c98cfa84
running : 2268th. #dcaebea9c5cd8a61d4205f67ae2d5e76
running : 2269th. #37cb9783fb11bd0c244bc9c0dcdfe124
running : 2270th. #4d9f744695a2bacfbbe79d92cb07492c
running : 2271th. #99fe996c9f728f2ecac484a407b76491
running : 2272th. #ea374002e0601f83d7b8ce4d89881df5
running : 2273th. #a42573c1c4efb00e49f1141a68344969
running : 2274th. #9755e7da37e115514d1a114e3a2198eb
running : 2275th. #49d1b55cf4487bc953ebb9d70bb151fc
running : 2276th. #9f3c597d9fabe90c6ea7295415684d97
running : 2277th. #d0f44a369f46163c76011399fc73c43a
running : 2278th. #07029053288848a87dc21df9f0fed271
running : 2279th. #a95038b20ebc3f1b9a1dd3b6015fef34
running : 2280th. #31e318b0b855f5a801367f954dd2a718
running : 22

running : 2422th. #684153bb10b433b135513d2e1011b084
running : 2423th. #d33b656809fbab6cfaccc5b273e6d135
running : 2424th. #31553e01b10ff7089a1d392ed4e3f3ba
running : 2425th. #30e5df21ed8229a9d84728883e78d1d8
running : 2426th. #1da545a0de3dfee12dc400d918946c69
running : 2427th. #0c2cc4ad66cf88cb4a189a1a975f8921
running : 2428th. #a8a8a04886532cb39e3d04d9137d4b18
running : 2429th. #7f95d5d8b1d5c0397176bdf122f422a4
running : 2430th. #59bf83f9bf1a04c6bad4ddbed5a889a6
running : 2431th. #7a160cd8c265506a2c78c03920eafe08
running : 2432th. #ac9e0866396f99c72a9df37a81f0bd91
running : 2433th. #570679f4d09761889f9ab54bb0984023
running : 2434th. #5ec3e1c87f6a20598f1160cde4fb2407
running : 2435th. #f45dc39ee3e7554599a9fa7966be3f86
running : 2436th. #c3a2a592d4db5c86d0cb4219e5624cb7
running : 2437th. #f99cade24b9854e658cf414231a985fa
running : 2438th. #f1c086fb3d2090e1e92b631c4c9c0d22
running : 2439th. #c027b3ec7497ce58224e76a0900646e5
running : 2440th. #a72056349769f58c98568656dda94108
running : 24

running : 2581th. #c725e1241f1ef009cc748f451faccdbd
running : 2582th. #8cf1524eb187a54391ebd20cbe1c7487
running : 2583th. #f280c78f0c182f8eaa6f290e7ef0be73
running : 2584th. #5048681d1c88151096f2ab04bc9c9c77
running : 2585th. #97ff7a2ad0ba645437a179df21c4d1cf
running : 2586th. #ff74ac56393cbf8e0ce6305fc3273ed7
running : 2587th. #c35c6fd657df3f8f84af426a50fb8cfa
running : 2588th. #1c2880ba9a2c7d9fed30846c740d649c
running : 2589th. #f5b400a174a19381eb90706a1933e14b
running : 2590th. #2a5e90697c3b0b778cd468b0e8b06fd5
running : 2591th. #2dc96afc6cfa48ce7ded3758949d10d2
running : 2592th. #d7bb23e1e72ffc471836cc9fd9104529
running : 2593th. #ed063b24d8ed6905870a34017cbab314
running : 2594th. #dfa357ff206db6f8ba442c53b363c00d
running : 2595th. #705a9aa17625747c26b8ae7a2f3fafff
running : 2596th. #57677d5b85a9d177663d4cbc201f241e
running : 2597th. #76ca959bef49cb532dd024abb0d89dba
running : 2598th. #4fc02fb246bfa041ed02944481b30cca
running : 2599th. #fb67e7ad6fa1db55f904e04f0659510f
running : 26

running : 2746th. #1ec67d525816587046feec612c3e920b
running : 2747th. #8771efd77dbdfca59f7c6e9ea84701a6
running : 2748th. #7f2dc93f0f5253b74a51d06fa6d3aeef
running : 2749th. #adfad305b676259aefecf6eea8131332
running : 2750th. #23836b97dfccd14561c6d9cec49cc839
running : 2751th. #16d9ebd6d3fb19580cbdd990f05c3b1e
running : 2752th. #fa517887d0bb4d71df0d046fd8675b88
running : 2753th. #301f9be1fd3ac8822bcaf88098202f6c
running : 2754th. #f737a56b755a8a79747646a2b632da49
running : 2755th. #abb283e354b969f0f6787ce64ec2700a
running : 2756th. #49ca30c4e3e640ce94830227a04bed1c
running : 2757th. #8e0e0eba69611e2cdcb3404984d0171b
running : 2758th. #f7ea279f90f4fea2cc2b0e872e8fc48f
running : 2759th. #4207c636ce5e7bbcf25ae3cbecce7d29
running : 2760th. #c54fd96816d2b6c54c9a9be9b1374890
running : 2761th. #4baf705f04c3940217330ef11bed69be
running : 2762th. #23dabf8229ab7c761f1d26c4c49d1661
running : 2763th. #211f709942c4637e8f2208f8b5ec9eae
running : 2764th. #535c6366735646bea017c07e34e89b69
running : 27

running : 2908th. #bc63ee489cb2964b7766e8798759cad3
running : 2909th. #fcd234c0e05d0bea6aeae32fbe9e2db4
running : 2910th. #5cc6793c7c080a0ca4569af59d8afff0
running : 2911th. #59b24c5a827b9493747d1d1d2434f3d8
running : 2912th. #bb98e9376ad244a71b5de67081fa5860
running : 2913th. #8e9d512519ee10ee0c6ca0d16c865dc0
running : 2914th. #6d6487dc8fd58037ec6bd9dd528fc5b2
running : 2915th. #dac7311b11f1f8f5615bc4eb51538d08
running : 2916th. #a111da43e052d27ae527a0d439751be4
running : 2917th. #86099a3530b6b80837085a15926d1edf
running : 2918th. #13d15dbe1d3fa21359b1d221731e7f75
running : 2919th. #4eb18e90c9bc3122b7f8424aad47762d
running : 2920th. #4b0824bc3b78df120fad874dca5fbfec
running : 2921th. #3a2ef692baf0ad9c23e85d2fe0d3b80d
running : 2922th. #8ddc4122911c68e16fb4ad42ccfb924c
running : 2923th. #59185c3832e818157922f7b8db119602
running : 2924th. #27cdb299806bc26a03882d7be30d4092
running : 2925th. #384c90be5b369636264c70bff813d30c
running : 2926th. #b64eb97bc9bc5e51c38968e99e21ef0c
running : 29

In [10]:
# sorted
'''

import operator
count = {}
for user in user_read_list :
    count[len(user_read_list[user])] = count.get(len(user_read_list[user]), 0) + 1
    
count = sorted(count.items(), key=operator.itemgetter(0))

pprint(count)
'''

[(1, 35837),
 (2, 27337),
 (3, 17555),
 (4, 16295),
 (5, 11138),
 (6, 10769),
 (7, 8324),
 (8, 8334),
 (9, 6403),
 (10, 6511),
 (11, 5389),
 (12, 5339),
 (13, 4367),
 (14, 4363),
 (15, 3920),
 (16, 4019),
 (17, 3304),
 (18, 3409),
 (19, 3000),
 (20, 2920),
 (21, 2596),
 (22, 2642),
 (23, 2447),
 (24, 2369),
 (25, 2084),
 (26, 2184),
 (27, 1909),
 (28, 1965),
 (29, 1748),
 (30, 1770),
 (31, 1656),
 (32, 1616),
 (33, 1469),
 (34, 1440),
 (35, 1385),
 (36, 1415),
 (37, 1317),
 (38, 1295),
 (39, 1242),
 (40, 1243),
 (41, 1110),
 (42, 1082),
 (43, 1014),
 (44, 1149),
 (45, 971),
 (46, 998),
 (47, 905),
 (48, 933),
 (49, 940),
 (50, 835),
 (51, 813),
 (52, 848),
 (53, 752),
 (54, 773),
 (55, 756),
 (56, 786),
 (57, 696),
 (58, 692),
 (59, 647),
 (60, 648),
 (61, 682),
 (62, 647),
 (63, 644),
 (64, 607),
 (65, 642),
 (66, 594),
 (67, 568),
 (68, 541),
 (69, 557),
 (70, 529),
 (71, 546),
 (72, 559),
 (73, 493),
 (74, 499),
 (75, 503),
 (76, 468),
 (77, 482),
 (78, 471),
 (79, 493),
 (80, 401),

 (2823, 1),
 (2825, 1),
 (2826, 1),
 (2828, 1),
 (2833, 1),
 (2834, 1),
 (2838, 1),
 (2842, 1),
 (2844, 1),
 (2845, 1),
 (2853, 1),
 (2854, 1),
 (2855, 1),
 (2857, 1),
 (2860, 1),
 (2868, 1),
 (2869, 1),
 (2875, 1),
 (2878, 1),
 (2882, 1),
 (2887, 1),
 (2891, 1),
 (2892, 1),
 (2900, 1),
 (2901, 1),
 (2905, 1),
 (2912, 1),
 (2914, 1),
 (2918, 2),
 (2919, 1),
 (2924, 2),
 (2926, 2),
 (2927, 2),
 (2928, 1),
 (2930, 1),
 (2932, 1),
 (2936, 1),
 (2937, 1),
 (2939, 2),
 (2941, 1),
 (2942, 1),
 (2967, 1),
 (2968, 1),
 (2971, 1),
 (2974, 1),
 (2976, 1),
 (2977, 1),
 (2978, 1),
 (2981, 1),
 (2982, 1),
 (2988, 1),
 (2995, 2),
 (2996, 2),
 (3003, 1),
 (3004, 1),
 (3009, 2),
 (3012, 1),
 (3015, 1),
 (3020, 2),
 (3021, 1),
 (3022, 1),
 (3023, 1),
 (3025, 1),
 (3026, 2),
 (3028, 1),
 (3032, 1),
 (3036, 2),
 (3037, 1),
 (3038, 1),
 (3039, 1),
 (3040, 1),
 (3042, 1),
 (3044, 1),
 (3046, 1),
 (3050, 1),
 (3054, 2),
 (3055, 2),
 (3056, 1),
 (3064, 1),
 (3065, 1),
 (3067, 1),
 (3068, 1),
 (3069, 1),
 (30